In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Ruta general
base_dir = 'Coleccion/'

#Sub rutas Modelos
m1 = base_dir + 'Disco_compresion__Carga_Lineal_3500N__100_frames__256_Resolucion/'
m2 = base_dir + 'Anillo_compresion__Carga_Lineal_3200N__100_frames__256_Resolucion/'
m3 = base_dir + 'Placa_Infinita__Carga_Lineal_7000N__100_frames__256_Resolucion/'
m4 = base_dir + 'Viga_distribuida__Carga_Lineal_3000N__100_frames__256_Resolucion/'
m5 = base_dir + 'Viga_puntual__Carga_Lineal_3000N__100_frames__256_Resolucion/'

ims = 'Imagenes/'

# Sub Rutas iluminacion + camara
c = ['Concord_IMX250MYR_No_correccion/',
 'Concord_IMX250MZR_No_correccion/',
 'DL11_IMX250MYR_No_correccion/',
 'DL11_IMX250MZR_No_correccion/',
 'Fluorescente_IMX250MYR_No_correccion/',
 'Fluorescente_IMX250MZR_No_correccion/',
 'LCD_IMX250MYR_No_correccion/',
 'LCD_IMX250MZR_No_correccion/',
 'Philips_IMX250MYR_No_correccion/',
 'Philips_IMX250MZR_No_correccion/',
 'RXD2_IMX250MYR_No_correccion/',
 'RXD2_IMX250MZR_No_correccion/',
 'Sylvania_IMX250MYR_No_correccion/',
 'Sylvania_IMX250MZR_No_correccion/',
 'Toshiba_IMX250MYR_No_correccion/',
 'Toshiba_IMX250MZR_No_correccion/',
 'Willard_IMX250MYR_No_correccion/',
 'Willard_IMX250MZR_No_correccion/']

cs = ['3', '4', '10', '11', '17', '18', '24', '25', '31', '32', '38', '39', '45', '46', '52', '53', '59', '60'] 
# Sub Rutas imagenes originales
i1 = 'Img_1/'
i2 = 'Img_2/'
i3 = 'Img_3/'
i4 = 'Img_4/'

# Almacenamiento de stacks
folder = 'Stokes/'

mm = [m1, m2, m3, m4, m5]
ii = [i1, i2, i3, i4]

In [10]:
dictionary = {}
for m in mm:
    for z in c:
        for i in range(1, 501):
            key = m + ims + z + folder + 'Stokes_M' + str(mm.index(m) + 1) + '_' + 'C' + str(cs[c.index(z)]) + '_' + str(i) + '.npy'
            value = m + 'PNA' + str(mm.index(m) + 1) + '/' + 'PN_' + str(i) + '.txt'
            dictionary[key] = value

In [11]:
df = pd.DataFrame(list(dictionary.items()), columns=['stackURL', 'phaseURL'])
df.head()

,stackURL,phaseURL
0,Coleccion/Disco_compresion__Carga_Lineal_3500N...,Coleccion/Disco_compresion__Carga_Lineal_3500N...
1,Coleccion/Disco_compresion__Carga_Lineal_3500N...,Coleccion/Disco_compresion__Carga_Lineal_3500N...
2,Coleccion/Disco_compresion__Carga_Lineal_3500N...,Coleccion/Disco_compresion__Carga_Lineal_3500N...
3,Coleccion/Disco_compresion__Carga_Lineal_3500N...,Coleccion/Disco_compresion__Carga_Lineal_3500N...
4,Coleccion/Disco_compresion__Carga_Lineal_3500N...,Coleccion/Disco_compresion__Carga_Lineal_3500N...


In [12]:
len(df['stackURL'].unique())

45000

In [13]:
len(df['phaseURL'].unique())

2500

In [14]:
df['stackURL'][1000]

'Coleccion/Disco_compresion__Carga_Lineal_3500N__100_frames__256_Resolucion/Imagenes/DL11_IMX250MYR_No_correccion/Stokes/Stokes_M1_C10_1.npy'

In [15]:
df['phaseURL'][1000]

'Coleccion/Disco_compresion__Carga_Lineal_3500N__100_frames__256_Resolucion/PNA1/PN_1.txt'

## **Creación de datasets de imágenes de entrada y salida.**

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['stackURL'], df['phaseURL'],test_size=0.2)

**Construcción arrays de ENTRADA (S) de entrenamiento y validación con los valores de los 4-stack, correspondientes a los 4 estados de polarización de las imágenes:**

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36000,), (9000,), (36000,), (9000,))

In [11]:
##################### second part: create the h5py object #####################
import h5py

S_train_shape = (len(X_train), 256, 256, 3)
S_test_shape = (len(X_test), 256, 256, 3)
P_train_shape = (len(y_train), 256, 256)
P_test_shape = (len(y_test), 256, 256)


# open a hdf5 file and create earrays 
f = h5py.File("DatasetsStokes.hdf5", mode='w')

# PIL.Image: the pixels range is 0-255,dtype is uint.
# matplotlib: the pixels range is 0-1,dtype is float.
# the ".create_dataset" object is like a dictionary, the "train_labels" is the key.
f.create_dataset("S_train", S_train_shape)
f.create_dataset("S_test", S_test_shape)  
f.create_dataset("P_train", P_train_shape)
f.create_dataset("P_test", P_test_shape)

######################## third part: write the images #########################
import cv2

j = 0
# loop over S_train paths
for i in X_train.values:
    img = np.load(i)
    #img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)# resize to (128,128)
    f["S_train"][j, ...] = img
    
    if j % 1000 == 0:
        print ('S_Train data: {}/{}'.format(j, len(X_train)))
    
    j += 1
        
j = 0
# loop over S_test paths
for i in X_test:

    img = np.load(i)
    #img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)# resize to (128,128)
    f["S_test"][j, ...] = img 
    
    if j % 1000 == 0:
        print ('S_Test data: {}/{}'.format(j, len(X_test)))
    
    j +=1

j = 0
# loop over P_train paths
for i in y_train:
  
    img = np.loadtxt(i)
    #img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)# resize to (128,128)
    f["P_train"][j, ...] = img 
    
    if j % 1000 == 0:
        print ('P_Train data: {}/{}'.format(j, len(y_train)))
    
    j += 1

j = 0
# loop over P_test paths
for i in y_test:

    img = np.loadtxt(i)
    #img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)# resize to (128,128)
    f["P_test"][j, ...] = img 
    
    if j % 1000 == 0:
        print ('P_Test data: {}/{}'.format(j, len(y_test)))
    
    j += 1
        
f.close()

S_Train data: 0/7200
S_Train data: 1000/7200
S_Train data: 2000/7200
S_Train data: 3000/7200
S_Train data: 4000/7200
S_Train data: 5000/7200
S_Train data: 6000/7200
S_Train data: 7000/7200
S_Test data: 0/1800
S_Test data: 1000/1800
P_Train data: 0/7200
P_Train data: 1000/7200
P_Train data: 2000/7200
P_Train data: 3000/7200
P_Train data: 4000/7200
P_Train data: 5000/7200
P_Train data: 6000/7200
P_Train data: 7000/7200
P_Test data: 0/1800
P_Test data: 1000/1800


In [ ]:
ls

In [ ]:
for i in X_train.values:
    print(i)

In [ ]:
res = np.load(X_train[2])
img = cv2.resize(res, (128, 128), interpolation=cv2.INTER_AREA)

In [ ]:
plt.imshow(img, cmap='gray')

In [ ]:
img.shape

In [ ]:
X_train[0]

In [ ]:
f.close()

In [ ]:
import h5py
f = h5py.File("ensayo5.hdf5", mode='w')
f.create_dataset("S_train", (1, 128, 128, 4))


In [ ]:
stack = np.load(X_train[67317])
img = cv2.resize(stack, (128, 128), interpolation=cv2.INTER_AREA)
f["S_train"][0, ...] = img

In [ ]:
hf = h5py.File('ensayo5.hdf5', 'r')

In [ ]:
n1 = hf.get('S_train')
n1

In [ ]:
n1 = np.array(n1)
n1.shape

In [ ]:
n1[0,:,:,:].shape

In [ ]:
import numpy as np

S_train = []
i= 1

for path in X_train.values:
    stack = np.load(path)
    S_train.append(stack)
    print('\r' + str(i), end='')
    i += 1
    
S_train = np.array(S_train)

In [ ]:
S_train.shape

In [ ]:
np.save('/Coleccion/NumpyArrays/S_train', S_train)

## Validación

In [ ]:
S_test = []
i= 1

for path in X_test.values:
    stack = np.load(path)
    S_test.append(stack)
    print('\r' + str(i), end='')
    i += 1
    
S_test = np.array(S_test)

In [ ]:
stack = np.loadtxt(y_train.values[0])

In [ ]:
stack.shape

In [ ]:
plt.imshow(stack, cmap='gray')

In [ ]:
stack.shape

In [ ]:
S_test = []
S_test.append(stack)
S_test = np.array(S_test)

In [ ]:
np.savetxt('S_test.txt', S_test[0,:,:,0])

In [ ]:
S_test.shape

In [ ]:
np.save('S_test', S_test)

In [ ]:
a1 = np.loadtxt('S_test.txt')
a1.shape

In [ ]:
Matrix = np.load(X_train.values[1])
Matrix.shape

In [ ]:
plt.imshow(Matrix)

In [ ]:
f = open('S_test.txt', 'a')
f.write(str(S_test[0,:,:,1]))
f.close()

In [ ]:
a2 = np.load('S_test.npy')
a2.shape

In [ ]:
plt.imshow(a2[0,:,:,:])

In [ ]:
from pathlib import Path

In [ ]:
p = Path('S_test.npy')

with p.open('ab') as f:
    np.save(f, Matrix)